# Multidimensional Array Exercise

In this exercise, you will apply the principles of using multidimensional arrays to estimate global mean temperature from a gridded dataset, accounting for the fact that grid cells become smaller at the poles.

In a dataset with evenly spaced latitude and longitude values, the meridians converge at the poles, making the cells smaller.  The area of the cells is proportional to the cosine of the latitude $\theta$ (in radians):

$$ \Delta A = R_{\oplus}^2 \Delta \theta \Delta \phi \cdot \cos(\theta) $$
$$ \Delta A \propto \cos(\theta)$$

Therefore, when calculating the global average of a quantity (like temperature), the values need to be weighted by $\cos(\theta)$.

**Goal**: correctly calculate a timeseries of global mean temperature from the dataset below

Instructions:

1. execute the cell below to download the dataset (this is the one we used to make sound in the first class)
1. the cell includes code to extract the latitude values (in degrees; `lat`) and the temperature field `temp_3d` as numpy arrays
1. calculate and plot a timeseries of global mean temperature, using the $cos(\theta)$ weighting

In [5]:
""" Read in the data """
import xarray as xr

# set the year we want to download
year = 1983 
# set the URL for the NCEP/DOE Reanalysis 2 data file
url = f"https://psl.noaa.gov/thredds/fileServer/Datasets/ncep.reanalysis2/gaussian_grid/air.2m.gauss.{year}.nc"

# set the name of the file we want to download to
output_file = f"air.2m.gauss.{year}.nc"

# download the data file
# NOTE: the use of ! at the beginning of the line indicates that this is a shell command, not python code -- though it does use some python code.  How, why?
# check first if the file exists; don't re-download if it does
import os
if not os.path.exists(output_file):
    ! curl --output {output_file} {url}

# (a side note for anyone familiar with xarray: you might ask why I don't use xarray to directly open the file from the URL (or the related OpenDAP URL)?  The reason is that it takes several minutes to open this 55 MB file, whereas directly downloading it takes only a couple seconds!)

# open the dataset using xarray
temp_ds = xr.open_dataset(output_file, chunks = -1)

# get the latitude and temperature values as numpy arrays
lat = temp_ds.lat.values
temp_3d = temp_ds.air.values

First, convert latitude to raidans. Then convert latitude to cosine (because it is proportional)
This data may not give accurate area dimensions, but it will be proportional to area. 

In [6]:
#import numpy
import numpy as np

#convert to radians
lat_rad = lat*np.pi/180 #hope this worked correctly

#get cos of latitudes
cos_lat = np.cos(lat_rad)

#check
print(lat[:10])#shows first 10 values
print(lat_rad[:10])#shows first 10 values
print(cos_lat[:10])#shows first 10 values


[88.542  86.6531 84.7532 82.8508 80.9473 79.0435 77.1394 75.2351 73.3307
 71.4262]
[1.5453495 1.512382  1.4792224 1.4460193 1.4127971 1.3795694 1.3463365
 1.3131002 1.2798623 1.2466223]
[0.0254441  0.05838108 0.09144598 0.12445351 0.15734267 0.19006358
 0.22257979 0.25485343 0.28684714 0.31852594]


We want something proportional to the area. Longitude spacing is constant(?) so latitude will work.

Multiply temp values by cosine of latitude to weight temperature data. Which dimension do we multiply?

In [13]:
temp_ds

<xarray.Dataset>
Dimensions:  (level: 1, lat: 94, lon: 192, time: 1460)
Coordinates:
  * level    (level) float32 2.0
  * lat      (lat) float32 88.54 86.65 84.75 82.85 ... -84.75 -86.65 -88.54
  * lon      (lon) float32 0.0 1.875 3.75 5.625 7.5 ... 352.5 354.4 356.2 358.1
  * time     (time) datetime64[ns] 1983-01-01 ... 1983-12-31T18:00:00
Data variables:
    air      (time, level, lat, lon) float32 dask.array<chunksize=(1460, 1, 94, 192), meta=np.ndarray>
Attributes:
    Conventions:    CF-1.0
    title:          4x Daily NCEP/DOE Reanalysis 2
    comments:       Data is from \nNCEP/DOE AMIP-II Reanalysis (Reanalysis-2)...
    platform:       Model
    source:         NCEP/DOE AMIP-II Reanalysis (Reanalysis-2) Model
    institution:    National Centers for Environmental Prediction
    dataset_title:  NCEP-DOE AMIP-II Reanalysis
    References:     https://www.psl.noaa.gov/data/gridded/data.ncep.reanalysi...
    source_url:     http://www.cpc.ncep.noaa.gov/products/wesley/reanalysis2/
    history:        created 2002/03 by RHS (netCDF2.3)\nConverted to chunked,...

The air data is what is extracted in the temp_3d variable.

In [28]:
temp_ds_array = np.zeros((1,94,192,1460), dtype = 'd')
#temp_3d = temp_ds.mean(axis = 0)
temp_ds_array = temp_ds["level", "lat", "lon", "time"]
temp_3d = temp_ds_array[0, :, :, :]
print(temp_3d.shape())


KeyError: ('level', 'lat', 'lon', 'time')

In [9]:
weight_temp = temp_3d[]*lat_rad
print(weight_temp)

In [10]:
#normalize weights*****

area_wgts = area_wgts / area_wgts.sum()

NameError: name 'area_wgts' is not defined

In [ ]:
area_wgts = cos_lat[np.newaxis, :, np.newaxis]
area_wgts.shape #94 corrseponds to latitude

(1, 94, 1)

weighted sum = sum of weights * variable / (sum of weights)

In [ ]:
temp_shape = temp_3d.shape

temp_weighted = temp_3d * area_wgts


In [ ]:

np.average(temp_3d, weights= area_wgts)

TypeError: Axis must be specified when shapes of a and weights differ.